In [ ]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import numpy as np
from IPython.display import clear_output, display
import sys
sys.path.insert(0,'..')
from celeba_reader import CelebaSequence

In [ ]:
paths = {
    'img_path': '/path/to/img_align_celeba',
    'attrs_path': '../list_attr_celeba.csv',
    'partition_path': '../list_eval_partition.csv'
}
batch = 100
test_data = CelebaSequence(partition=2, batch=batch, **paths)

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        tf.saved_model.loader.load(sess, [tag_constants.SERVING], './tmp')
        tf.summary.FileWriter( './logs/1/train', graph) # $ tensorboard --logdir ./logs/1
        images_batch = graph.get_tensor_by_name('Placeholder:0')
        labels = graph.get_tensor_by_name('Placeholder_1:0')
        classifier = graph.get_tensor_by_name('Reshape_1:0')
        
        correct = 0.
        total = 0.
        i = 0
        for x, y in test_data:
            total = total + len(y)
            predicted = sess.run(classifier, feed_dict={images_batch: x, labels: y})
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5 ] = 0
            correct = correct + np.sum(np.equal(y, predicted))
            clear_output()
            display(f"{correct}/{total}")
            if i == 40:
                break
            i = i + 1
        clear_output()
        display(f"accuracy={correct/total:.2f}")